<a href="https://colab.research.google.com/github/felipeSeeduc/consumo_consciente_colab/blob/main/Modelo_agua_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de libs

In [ ]:
!pip install wget

# Classes

## Classe DadosConsumoConsciente

In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,20)

from numpy import (array2string,
                   quantile,
                   polyfit)


class DadosConsumoConsciente:
  """Suite para determinação dos parâmetros de consumo para o projeto
  Consumo Consciente

  Inclui:

    - Tratamento, analise e visualização de dados;
    - Classificação de clusters com Kmeans;
    - Implementação da regra para determinação dos parâmetros de consumo
    das escola
    - Geração de arquivo .csv contendo o parâmetro para cada escola

  Parameters:
    model_name: nome definido para o modelo
    data_path: caminho para os dados
    model_data_filename: nome do arquivo usado para criação do modelo KMeans


  Attributes:
    DadosConsumoConsciente.ceja (list): lista com censos das escolas CEJA
    model_name (str): nome definido para o modelo.
    data_path (Path): caminho para os dados.
    model_data_filename (str): nome do arquivo usado para criação do modelo KMeans.
    model_raw_inputs (pd.DataFrame): DataFrame contendo as features para modelagem (ano anterior) de TODAS as escolas.
    model_consuptions (pd.DataFrameGroupBy): DataFrameGroupBy contendo os dados de consumos por escola, exceto as escolas CEJA
    mean_consumption (pd.DataFrame): DataFrame contendo as médias de consumos por escola, exceto as escolas CEJA
    measurements (pd.DataFrame): DataFrame contendo a quantidade de medidas de consumos
      aferidas por escola, exceto as escolas CEJA
    measurements_six_records_or_more (pd.DataFrame):  DataFrame contendo a quantidade de medidas de consumos
      aferidas por escola, exceto as escolas CEJA e quando a medidas for igual ou superior a 6
    model_consumption_outliers_free (pd.DataFrame): DataFrameGroupBy contendo os dados de consumos por escola, exceto as escolas CEJA e sem outliers
    mean_consumption_outliers_free (pd.DataFrame): DataFrame contendo as médias de consumos por escola, exceto as escolas CEJA e sem outliers
    measurements_outliers_free (pd.DataFrame): DataFrame contendo a quantidade de medidas de consumos
      aferidas por escola, exceto as escolas CEJA e sem outliers
    measurements_six_records_or_more_outliers_free (pd.DataFrame):  DataFrame contendo a quantidade de medidas de consumos
      aferidas por escola, exceto as escolas CEJA e quando a medidas for igual ou superior a 6 e sem outliers
    kmeans_schools (pd.DataFrame): Escolas com dados disponibilizados na planilha de treinamento.
    schools_with_calculated_mean (pd.DataFrame): Lista de escolas com média calculada
    schools_with_six_records_or_more (pd.DataFrame): Lista de escolas com quantidade de medidas de consumos aferidas é igual ou superior a 6
    schools_with_six_records_or_more_outliers_free (pd.DataFrame):
    selected_schools (pd.DataFrame): Lista de escolas selecionadas para treinar o modelo
    model_mean (pd.DataFrame): média de consumo por escola sobre os dados sobre as escolas
    com quantidade de medidas de consumos aferidas é igual ou superior a 6
    model_mean_outliers_free (pd.DataFrame): média de consumo por escola sobre os dados sobre as escolas
    com quantidade de medidas de consumos aferidas é igual ou superior a 6 e sem outliers
  """
  ceja =[33098972,33142297,33015163,33099774,33119740,
       33138699,33019665,33021643,33025088,33139245,
       33159602,33027080,33055300,33139970,33062889,
       33067678,33097925,33138575,33062862,33075042,
       33085404,33084025,33125295,33088691,33138753,
       33017999,33040060,33042390,33149380,33048274,
       33057982,33145199,33096848,33100250,33138834,
       33139830]

  def __init__(self,
               model_name: str,
               data_path: Path,
               model_data_filename: str
               ):

    self.ax = None
    self.model_name = model_name
    self.data_path = data_path
    self.model_data_filename = model_data_filename

    # CARGA DE DADOS PARA MODELAGEM (model_data_filename)
    ## PARA KMEANS
    self.model_raw_inputs = self.load_model_raw_inputs()
    ## CONSUMOS
    self.model_consuptions = self.load_model_consuptions()

    # CALCULOS SOBRE OS CONSUMOS
    ## CALCULA A MÉDIA DOS CONSUMOS SEM FILTRAGEM
    self.mean_consumption = self.set_mean_consumption(
        self.model_consuptions
    )

    ## CALCULA A QUANTIDADE DE MEDIDAS AFERIDAS PARA CADA ESCOLA SEM FILTRAGEM
    self.measurements = self.set_measurements(
        self.model_consuptions
    )

    ## DETERMINA AS ESCOLAS COM SEIS OU MAIS MEDIDAS AFERIDAS
    self.measurements_six_records_or_more = \
      self.measurements[
          self.measurements['MEDIDAS'] >= 6
          ]

    ## REMOVE OS OUTLIERS QUANTO AO CONSUMO
    self.model_consuptions_outliers_free = \
      self.remove_outliers(self.model_consuptions.obj, "MEDIDA").groupby("CENSO")

    ## CALCULA A MEDIA DO CONJUNTO SEM OUTLIERS
    self.mean_consumption_outliers_free = self.set_mean_consumption(
        self.model_consuptions_outliers_free
    )

    ## CALCULA A QUANTIDADE DE MEDIDAS AFERIDAS PARA CADA ESCOLA PARA O CONJUNTO SEM OUTLIERS
    self.measurements_outliers_free = self.set_measurements(
        self.model_consuptions_outliers_free
    )

    # DETERMINA AS ESCOLAS COM SEIS OU MAIS MEDIDAS AFERIDAS NO CONJUNTO SEM OUTLIERS
    self.measurements_six_records_or_more_outliers_free = \
      self.measurements_outliers_free[
          self.measurements_outliers_free['MEDIDAS'] >= 6
          ]


    # SELEÇÃO DE DADOS PARA O KMEANS
    ## TODAS AS ESCOLAS COM FEATURES DECLARADAS
    self.kmeans_schools = self.set_kmeans_schools()
    ## LISTA DE ESCOLAS PARA FILTRAGEM
    self.schools_with_calculated_mean = self.mean_consumption["CENSO"]
    self.schools_with_six_records_or_more = self.measurements_six_records_or_more["CENSO"]
    self.schools_with_six_records_or_more_outliers_free = \
      self.measurements_six_records_or_more_outliers_free["CENSO"]
    ## ESCOLAS SELECIONADAS PARA O MODELO
    self.selected_schools = self.set_selected_schools()


    self.model_mean = self.set_model_mean()
    self.model_mean_outliers_free = self.set_model_mean_outliers_free()

  def load_model_raw_inputs(self) -> pd.DataFrame :
    """
    Lê o arquivo 'model_data_filename' e retorna o DataFrame equivalente
    Returns:
      df_inputs_modelo_agua: DataFrame completo para o treinamento do modelo.
    """
    for file in self.data_path.iterdir():
      print(file.name)
      print(self.model_data_filename)
      if file.name == self.model_data_filename:
        df_inputs_modelo_agua = pd.read_excel(file, sheet_name="FEATURES")
        return df_inputs_modelo_agua

    return None

  def load_model_consuptions(self) -> pd.core.groupby.generic.DataFrameGroupBy:
    """
    Lê o arquivo 'model_data_filename' e retorna o DataFrameGroupBy por escola
    Returns:
      df_consumo: DataFrameGroupBy das informações relacionadas aos consumos por escola (CENSO).
    """
    for file in self.data_path.iterdir():
      if file.name == self.model_data_filename:
        df = pd.read_excel(file, sheet_name="CONSUMOS")
        df = df.loc[~df.CENSO.isin(DadosConsumoConsciente.ceja)]
        df_consumo = df.groupby("CENSO")
        return df_consumo


  def nonCEJA_features(self) -> pd.DataFrame:
    df_nonCEJA = self.model_raw_inputs.loc[~self.model_raw_inputs.CENSO.isin(DadosConsumoConsciente.ceja)]
    return df_nonCEJA


  def remove_outliers(self, df_in, col_name) -> pd.DataFrame:
    """Remoção de outliers baseado no padrão 1.5 do intervalo interquartílico

    Args:
      df_in: DataFrame a ser analisado
      col_name: coluna sobre a qual a remoção acontecerá

    Returns:
      df_out: DataFrame sem outliers
    """
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

  def set_mean_consumption(self, df: pd.core.groupby.generic.DataFrameGroupBy) -> pd.DataFrame:
    """Calcula a média de consumo por escola

    Args:
      df: DataFrameGroupBy por escola

    Returns:
      mean_consumption_df: médias de consumo por escola
    """
    mean_consumption_df = df['MEDIDA'].mean().reset_index()
    mean_consumption_df.rename(columns={"MEDIDA":"MEDIA_"+ self.model_name.upper().replace(" ", "_")}, inplace=True)
    return mean_consumption_df


  def set_model_mean(self) -> pd.DataFrame:
    """Calcula a média de consumo por escola sobre os dados sobre as escolas
    com quantidade de medidas de consumos aferidas é igual ou superior a 6

    Args:
      df: DataFrameGroupBy por escola

    Returns:
      mean_consumption_df: médias de consumo por escola
    """

    medias_inputs = self.model_raw_inputs.groupby("CENSO").sum()
    medias_agua =self.model_consuptions['MEDIDA'].mean()\
        .loc[self.schools_with_six_records_or_more]
    medias_inputs = medias_inputs.merge(medias_agua, left_index=True, right_index=True)
    return medias_inputs


  def set_model_mean_outliers_free(self)-> pd.DataFrame:
    """Calcula a média de consumo por escola sobre os dados sobre as escolas
    com quantidade de medidas de consumos aferidas é igual ou superior a 6
    e sem os outliers

    Args:
      df: DataFrameGroupBy por escola

    Returns:
      mean_consumption_df: médias de consumo por escola
    """

    medias_inputs_sem_outliers = self.model_raw_inputs.groupby("CENSO").sum()
    medias_agua_sem_outliers =self.model_consuptions_outliers_free['MEDIDA'].mean()\
        .loc[self.schools_with_six_records_or_more_outliers_free]
    medias_inputs_sem_outliers = medias_inputs_sem_outliers.merge(medias_agua_sem_outliers,
                                                                  left_index=True,
                                                                  right_index=True)
    return medias_inputs_sem_outliers


  def set_measurements(self, df: pd.DataFrame) ->pd.DataFrame:
    """Calcula a quantidade de medidas de consumos aferidas

    Args:
      df: DataFrameGroupBy por escola

    Returns:
      measurements_df: a quantidade de medidas de consumos aferidas
    """

    measurements_df = df['MEDIDA'].count().reset_index()
    measurements_df.rename(columns={"MEDIDA":"MEDIDAS"}, inplace=True)
    return measurements_df

  def set_kmeans_schools(self) -> pd.DataFrame:
    """Lista de escolas com dados disponibilizados na planilha de treinamento

    Returns:
      kmeans_schools: Escolas listadas na planilha de treinamento
    """
    kmeans_schools = self.model_raw_inputs['CENSO']
    return kmeans_schools


  def set_selected_schools(self):
    """Lista de escolas selecionadas para treinar o modelo
    Returns:
      kmeans_schools: Escolas listadas na planilha de treinamento
    """
    selected_schools = self.kmeans_schools.isin(
        self.schools_with_six_records_or_more
    )
    return selected_schools

  def greatest_measure(self):
    print("Escola com maior medida aferida: \n{0}".format(
    self.model_consuptions.max().sort_values("MEDIDA", ascending=False).max()))

  def count_of_schools_with_calculated_mean(self):
    print("Escolas ao menos um registro: {0}".format(
        self.schools_with_calculated_mean.count()))

  def count_of_schools_with_six_measurements_or_more(self):
    print("Escolas: {0} \nObservações: {1}".format(
    self.schools_with_six_records_or_more.count(),
    self.measurements_six_records_or_more["MEDIDAS"].sum())
     )

  # MÉTODOS PARA VISUALIZAÇÃO/PLOTAGEM
  def plot_consumptions_histogram(self):
    if self.ax:
      plt.close(self.ax.figure)

    medidas_validas = self.model_consuptions.obj["CENSO"].isin(self.schools_with_six_records_or_more)
    valor_maximo = self.model_consuptions["MEDIDA"].max().max()
    self.ax = self.model_consuptions.obj[medidas_validas]["MEDIDA"].plot.hist(bins= 80,
                                                                  edgecolor="red",
                                                                  facecolor="blue"
                                                                  # density=True
                                                                )
    self.ax.set_ylabel("Frequência")
    self.ax.set_xlabel("Consumos")
    self.ax.set_box_aspect(3/4)
    self.ax.figure
    plt.show()

  def plot_consumptions_boxplot(self, measure_unit: str) -> None:
    if self.ax:
      plt.close(self.ax.figure)
    escolas_com_medidas_validas = self.model_consuptions.obj["CENSO"].isin(
        self.schools_with_six_records_or_more)
    medidas_validas = self.model_consuptions.obj[escolas_com_medidas_validas][["MEDIDA"]]
    valor_maximo = medidas_validas.max().max()
    valor_minimo = medidas_validas.min().min()
    valor_medio = medidas_validas.mean().mean()

    quartis = quantile(medidas_validas, [0.25,0.5,0.75])
    iq = quartis[-1] -quartis[0]
    b_1, b_2 = max(valor_minimo,quartis[0] - 1.5*iq), min(valor_maximo,quartis[-1] + 1.5*iq)
    quartis_txt = array2string(quartis,
                              suppress_small=True,
                              separator=", ",
                              formatter={'float_kind':lambda x: "%.1f" % x})
    quartis_txt

    self.ax = medidas_validas.boxplot(
                                grid=True,
                                vert=False,
                                # patch_artist=True
                                )
    self.ax.set_mouseover(True)
    self.ax.set_xlabel("Consumo em " + measure_unit)
    self.ax.set_yticklabels([""])
    self.ax.set_box_aspect(3/4)
    msg = "Minimo: {0:.2f}{7}\n\
    Máximo: {1:.2f}{7}\n\
    Média: {2:.2f}{7}\n\
    Quatis: {3}\n\
    IQ: {4:.2f}{7}\n\
    B_min: {5:.2f}{7}\n\
    B_max: {6:.2f}{7}\n"\
            .format(valor_minimo,
                    valor_maximo,
                    valor_medio,
                    quartis_txt,
                    iq,
                    b_1,
                    b_2,
                    measure_unit
                  )
    txt = self.ax.text(3000,1.1,
                  msg,
                  horizontalalignment="center",
                  bbox ={"edgecolor":"black",
                        "facecolor":"yellow"}
                )
    plt.show()

  def plot_mean_consumption_histogram(self):
    if self.ax:
      plt.close(self.ax.figure)

    medias_agua =self.model_consuptions['MEDIDA'].mean()\
        .loc[self.schools_with_six_records_or_more]
    self.ax = medias_agua.plot.hist(bins= 80,
                              edgecolor="red",
                              facecolor="blue"
                              )
    self.ax.set_ylabel("Frequência")
    self.ax.set_xlabel("Consumos")
    self.ax.set_box_aspect(3/4)
    plt.show()

  def plot_mean_consumption_boxplot(self, measure_unit) -> None:
    if self.ax:
      plt.close(self.ax.figure)

    medias_agua = self.model_consuptions['MEDIDA'].mean()\
      .loc[self.schools_with_six_records_or_more]
    valor_maximo = medias_agua.max()
    valor_minimo = medias_agua.min()
    valor_medio = medias_agua.mean()

    quartis = quantile(medias_agua, [0.25,0.5,0.75])
    iq = quartis[-1] -quartis[0]
    b_1, b_2 = max(valor_minimo,quartis[0] - 1.5*iq), min(valor_maximo,quartis[-1] + 1.5*iq)
    quartis_txt = array2string(quartis,
                              suppress_small=True,
                              separator=", ",
                              formatter={'float_kind':lambda x: "%.1f" % x})

    self.ax = medias_agua.plot.box(
                                grid=True,
                                vert=False,
                                # patch_artist=True
                                )
    self.ax.set_mouseover(True)
    self.ax.set_xlabel("Consumo em "+ measure_unit)
    self.ax.set_yticklabels([""])
    self.ax.set_box_aspect(3/4)
    msg = "Minimo: {0:.2f}{7}\n\
    Máximo: {1:.2f}{7}\n\
    Média: {2:.2f}{7}\n\
    Quatis: {3}\n\
    IQ: {4:.2f}{7}\n\
    B_min: {5:.2f}{7}\n\
    B_max: {6:.2f}{7}\n"\
            .format(valor_minimo,
                    valor_maximo,
                    valor_medio,
                    quartis_txt,
                    iq,
                    b_1,
                    b_2,
                    measure_unit
                  )
    txt = self.ax.text( 1750,1.1,
                  msg,
                  horizontalalignment="center",
                  bbox ={"edgecolor":"black",
                        "facecolor":"yellow"}
                )
    plt.show()

  def plot_mean_consumption_outliers_free_histogram(self):
    if self.ax:
      plt.close(self.ax.figure)

    medias_agua_sem_outliers =self.model_consuptions_outliers_free['MEDIDA'].mean()\
        .loc[self.schools_with_six_records_or_more_outliers_free]
    self.ax = medias_agua_sem_outliers.plot.hist(bins= 80,
                              edgecolor="red",
                              facecolor="blue"
                              )
    self.ax.set_ylabel("Frequência")
    self.ax.set_xlabel("Consumos")
    self.ax.set_box_aspect(3/4)
    plt.show()

  def plot_mean_consumption_outliers_free_bloxplot(self, measure_unit):
    if self.ax:
      plt.close(self.ax.figure)

    medias_agua_outliers_free = self.model_consuptions_outliers_free['MEDIDA'].mean()\
      .loc[self.schools_with_six_records_or_more_outliers_free]
    valor_maximo = medias_agua_outliers_free.max()
    valor_minimo = medias_agua_outliers_free.min()
    valor_medio = medias_agua_outliers_free.mean()

    quartis = quantile(medias_agua_outliers_free, [0.25,0.5,0.75])
    iq = quartis[-1] -quartis[0]
    b_1, b_2 = max(valor_minimo,quartis[0] - 1.5*iq), min(valor_maximo,quartis[-1] + 1.5*iq)
    quartis_txt = array2string(quartis,
                              suppress_small=True,
                              separator=", ",
                              formatter={'float_kind':lambda x: "%.1f" % x})

    self.ax = medias_agua_outliers_free.plot.box(
                                grid=True,
                                vert=False,
                                # patch_artist=True
                                )
    self.ax.set_mouseover(True)
    self.ax.set_xlabel("Consumo em "+measure_unit )
    self.ax.set_yticklabels([""])
    self.ax.set_box_aspect(3/4)
    msg = "Minimo: {0:.2f}{7}\n\
    Máximo: {1:.2f}{7}\n\
    Média: {2:.2f}{7}\n\
    Quatis: {3}\n\
    IQ: {4:.2f}{7}\n\
    B_min: {5:.2f}{7}\n\
    B_max: {6:.2f}{7}\n"\
            .format(valor_minimo,
                    valor_maximo,
                    valor_medio,
                    quartis_txt,
                    iq,
                    b_1,
                    b_2,
                    measure_unit
                  )
    txt = self.ax.text( 250,1.1,
                  msg,
                  horizontalalignment="center",
                  bbox ={"edgecolor":"black",
                        "facecolor":"yellow"}
                )
    plt.show()

  def plot_scatter_measurement(self, x_name: str):

    if self.ax:
      plt.close(self.ax.figure)
    y, x = self.model_mean.MEDIDA, self.model_mean[x_name]
    valor_medio = y.mean()
    a, b = polyfit(x, y, 1)

    self.ax = self.model_mean.plot.scatter(y="MEDIDA", x=x_name, s=3)
    self.ax.plot(x, a*x+b, c="red")
    self.ax.plot(x, 0*x+valor_medio, c="black", ls=":")

    self.ax.set_ylabel("MEDIA")
    plt.grid()
    plt.show()

  def plot_scatter_measurement_outliers_free(self, x_name: str):

    if self.ax:
      plt.close(self.ax.figure)
    y, x = self.model_mean_outliers_free.MEDIDA, self.model_mean_outliers_free[x_name]
    valor_medio = y.mean()
    a, b = polyfit(x, y, 1)

    self.ax = self.model_mean_outliers_free.plot.scatter(y="MEDIDA", x=x_name, s=3)
    self.ax.plot(x, a*x+b, c="red")
    self.ax.plot(x, 0*x+valor_medio, c="black", ls=":")

    self.ax.set_ylabel("MEDIA")
    plt.grid()
    plt.show()

  def get_particular_feature(self) -> str:
    """TODO
    Refatorar...
    """
    if self.model_name == "Agua":
      return "SAIDAS_AGUA"
    if self.model_name == "Energia":
      return "SALAS_CLIMATIZADAS"
    else:
      return "Nenhum feature particular foi definido"


  # MÉTODOS PARA CORRELAÇÃO
  def corr(self):
    return self.model_mean[["MEDIDA", "TOTAL_ESTUDANTES", "TOTAL_AMBIENTES", self.get_particular_feature()]].corr(method='pearson')

  def corr_outliers_free(self):
    return self.model_mean_outliers_free[["MEDIDA", "TOTAL_ESTUDANTES", "TOTAL_AMBIENTES", self.get_particular_feature()]].corr(method='pearson')


## Classe ModeloConsumoConscienteKmeans

In [ ]:
from matplotlib.cm import Accent
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

from numpy import (unique,
                   array2string)

class ModeloConsumoConsciente:
  """ Class para treinamento do kmeans e aplicação da metodologia da parametrização

  Args:
    inputs (ModeloConsumoConsciente): dados tratados e analizados,
    n_clusters (int): quantidade de clusters do modelo

  Attributes:
    inputs:
    filtered_inputs:
    space:
    n_clusters:
    pipeline:
    clusters:
    scaler:
    kmeans:
    centroides:
    features:
    df_centroides:
    df_with_clusters_and_mean
    init_df_for_analisys
    median
    q1
    sigma
    mean_cluster
    df_for_analisys


  """

  def __init__(self, inputs: DadosConsumoConsciente, n_clusters = 10):
    self.inputs = inputs
    self.model_name_upper = self.inputs.model_name.upper().replace(" ", "_")
    self.particular_feature = self.inputs.get_particular_feature()
    self.filtered_inputs = self.inputs.model_raw_inputs \
                .dropna(subset=["TOTAL_ESTUDANTES", self.particular_feature, "TOTAL_AMBIENTES"])\
                .loc[self.inputs.selected_schools]\
                .reset_index()\
                .drop(['index'], axis=1)
    self.space = self.filtered_inputs.loc[:, ["TOTAL_ESTUDANTES", self.particular_feature, "TOTAL_AMBIENTES"]]
    self.n_clusters = n_clusters
    self.pipeline = self.set_kmeans_pipeline()

    self.clusters = self.pipeline.fit_predict(self.space)


    self.scaler= self.pipeline[0]
    self.kmeans = self.pipeline[-1]
    self.centroides =  self.scaler.inverse_transform(self.kmeans.cluster_centers_).round(0)
    self.features = self.scaler.feature_names_in_
    self.df_centroides = pd.DataFrame(self.centroides,
                             columns = self.features,
                             index=(f"Cluster {c}" for c in range(self.centroides.shape[0])))\
                              .reset_index().rename(columns={"index":"CLUSTER"})

    self.df_with_clusters_and_mean = self.assign_kmeans_clusters()

    self.init_df_for_analisys = self.df_with_clusters_and_mean.loc[:, ["MEDIA_"+self.model_name_upper, "CLUSTER"]]
    self.median =  self.set_median()
    self.q1 =  self.set_q1()
    self.sigma =  self.set_sigma()
    self.mean_cluster =  self.set_mean_cluster()
    self.df_for_analisys = self.set_df_for_analisys()


  def set_median(self):
    medianas = self.init_df_for_analisys.groupby("CLUSTER").median().reset_index()
    medianas.rename(columns={"MEDIA_"+self.model_name_upper:"MEDIANA"}, inplace=True)
    return medianas

  def set_q1(self):
    q1 = self.init_df_for_analisys.groupby("CLUSTER").quantile(0.25).reset_index()
    q1.rename(columns={"MEDIA_"+self.model_name_upper:"Q1"}, inplace=True)
    return q1

  def set_sigma(self):
    sigma = self.init_df_for_analisys.groupby("CLUSTER").std().reset_index()
    sigma.rename(columns={"MEDIA_"+self.model_name_upper:"SIGMA"}, inplace=True)
    return sigma


  def set_mean_cluster(self):
    media_cluster = self.init_df_for_analisys.groupby("CLUSTER").mean().reset_index()
    media_cluster.rename(columns={"MEDIA_"+self.model_name_upper:"MEDIA_CLUSTER"}, inplace=True)
    return media_cluster

  def set_df_for_analisys(self):

    analise = self.init_df_for_analisys.merge(self.median).sort_values("MEDIANA").reset_index().drop("index", axis=1)
    analise = analise.merge(self.sigma).sort_values("MEDIANA").reset_index().drop("index", axis=1)
    analise = analise.merge(self.mean_cluster).sort_values("MEDIANA").reset_index().drop("index", axis=1)
    return analise



  def assign_kmeans_clusters(self):
    df = self.filtered_inputs.assign(CLUSTER = self.clusters.tolist())
    df = df.merge(self.inputs.mean_consumption)
    return df


  def plot_clusters(self, df, title, fig, subplot):

    ylim = df[self.particular_feature].max()
    ylabel = self.particular_feature


    ax = fig.add_subplot(subplot, projection='3d',
                        xlim = df.TOTAL_ESTUDANTES.max(),
                        ylim = ylim,
                        zlim = df.TOTAL_AMBIENTES.max(),
                        facecolor='white',
                       )
    ax.set_title(title)
    ax.set_xlabel('TOTAL_ESTUDANTES', fontsize= 15)
    ax.set_ylabel(ylabel,fontsize= 15)
    ax.set_zlabel('TOTAL_AMBIENTES', fontsize= 15)
    ax.invert_zaxis()
    ax.view_init(elev=15, azim=60, roll=0)
    return ax

  def set_kmeans_pipeline(self):
    scaler = StandardScaler()
    kmeans = KMeans(n_clusters=self.n_clusters, random_state=20)
    pipeline= make_pipeline(
                       scaler,
                        kmeans
                       )
    return pipeline

  def plot_clusters_(self):
    ## Plotando os resultados
    fig = plt.figure(
        figsize = (30,30)
    )
    ### Definição dos eixos
    ax_kmeans_agua = self.plot_clusters(self.space, "Kmeans Clusters - "+self.inputs.model_name, fig, 121)
    ax_centroides_agua = self.plot_clusters(self.space,
                                            "Kmeans Centroides -{self.inputs.model_name} \n (ESTUDANTES - SAIDAS DE AGUA - AMBIENTES)",
                                             fig, 122)

    ax= ax_centroides_agua.scatter(self.df_centroides.TOTAL_ESTUDANTES,
                          self.df_centroides[self.particular_feature],
                          self.df_centroides.TOTAL_AMBIENTES,
                          s = 100,
                          c = unique(self.clusters),
                          cmap='Dark2',
                          marker= "D",
                          alpha=1
                        )

    ax = ax_kmeans_agua.scatter(self.space.TOTAL_ESTUDANTES,
                          self.space[self.particular_feature],
                          self.space.TOTAL_AMBIENTES,
                          c = self.clusters,
                          s = 80,
                          cmap='Dark2',
                          alpha = 1
                    )


    for coords in self.df_centroides.values:
        x, y, z = coords[1:4]
        txt = f"({x:.1f},{y:.1f},{z:.1f})"
        txt_plot = ax_centroides_agua.text(x,y,z, txt, 'x',
                                      fontsize=10,
                                      horizontalalignment="center",
                                      verticalalignment="bottom"
                                    )

  def show_centroides(self):
    for i, c in enumerate(self.centroides):
      print("Cluster {0} - ALUNOS {1:5.0f} - {4}: {2:5.0f} - AMBIENTES {3:5.0f}".format(i, c[0], c[1], c[2], self.particular_feature))

  def plot_statistics(self):
    fig = plt.figure(
    figsize=(10,10)
    )
    ax_box = fig.add_subplot(221)
    ax_cluster = fig.add_subplot(222)
    ax_medias = fig.add_subplot(212)

    ax_box.yaxis.grid(True)
    ax_cluster.yaxis.grid(True)
    ax_medias.yaxis.grid(True)

    ## Recorte da base para análise
    analise = self.df_for_analisys

    medianas = self.median
    q1 = self.q1
    sigma = self.sigma
    media_cluster = self.mean_cluster

    groups_index = []
    labels = []

    for group, dataset in analise.groupby("MEDIANA"):
        groups_index.append(dataset.index)
        labels.append(dataset.CLUSTER.max())

    ## Grupos para plot
    for i, group in enumerate(groups_index):

        box = ax_cluster.boxplot(
                        analise.loc[group, ["MEDIA_"+self.model_name_upper]],
                        labels = [labels[i]],
                        positions= [i+1],
                        patch_artist = True,
                        meanline=True,
                        showmeans=True,
                        vert=False,
                        )

        box["boxes"][0].set_facecolor(Accent(labels[i]))


    analise['MEDIA_'+self.model_name_upper].plot(ax=ax_medias)
    analise.boxplot(ax=ax_box, column="MEDIA_"+self.model_name_upper)

    ax_box.set_title("BoxPlot sem Clusters - "+self.inputs.model_name)
    ax_cluster.set_title("BoxPlot Clusters - "+self.inputs.model_name)
    txt = ax_medias.set_title("Distribuição das Médias - "+self.inputs.model_name)

  def quartis_by_cluster(self):
    quartis = self.median
    quartis["CENTROIDE"] = ["".join(array2string(x, suppress_small=True)) for x in self.centroides]
    quartis.MEDIANA = quartis.MEDIANA.round(2)
    quartis = quartis.merge(self.q1)
    quartis.sort_values("CLUSTER")
    return quartis


  def show_training_results(self):
    resultado_final = self.df_with_clusters_and_mean.merge(self.median, on="CLUSTER")
    resultado_final = resultado_final.merge(self.sigma, on="CLUSTER")
    resultado_final = resultado_final.merge(self.mean_cluster, on="CLUSTER")
    resultado_final.rename(columns={"MEDIANA":"PARÂMETRO"}, inplace=True)
    return resultado_final


  def show_final_results(self):
    def set_parameter_callback(df:pd.DataFrame):
      sub_df = df[["Q1", "MEDIANA", "MEDIA_"+self.model_name_upper]]
      result = []
      for Q1, MEDIANA, MEDIA_AGUA in sub_df.values:
        if not pd.isna(MEDIA_AGUA):
            sub_result = Q1 if MEDIA_AGUA < Q1 else MEDIANA
        else:
            sub_result = Q1
        result.append(sub_result)
      return result

    inputs_set = self.inputs.nonCEJA_features().dropna(subset= ["TOTAL_ESTUDANTES", self.particular_feature, "TOTAL_AMBIENTES"])
    clusters = self.pipeline.predict(
        inputs_set.drop("CENSO", axis=1).loc[:, ["TOTAL_ESTUDANTES", self.particular_feature, "TOTAL_AMBIENTES"]]
        )

    inputs_set = inputs_set.assign(CLUSTER = clusters.tolist())
    inputs_set = inputs_set.merge(self.inputs.mean_consumption, how='left')

    centroides_str =["".join(array2string(x, suppress_small=True,separator=" - ")) for x in self.centroides]
    centroides = pd.DataFrame(centroides_str,columns=["CENTROIDES"]).reset_index().rename(columns={"index":"CLUSTER"})

    resultado_final = inputs_set.merge(self.median, on="CLUSTER")
    resultado_final = resultado_final.merge(self.sigma, on="CLUSTER")
    resultado_final = resultado_final.merge(centroides, on="CLUSTER")
    resultado_final = resultado_final.merge(self.mean_cluster, on="CLUSTER")
    resultado_final = resultado_final.merge(self.q1, on="CLUSTER")
    resultado_final = resultado_final.assign(PARAMETRO = set_parameter_callback)
    return resultado_final

  def save_final_result(self):
      filename = f'resultado_final_{self.model_name_upper.lower()}_{date.today().year}.csv'
      self.show_final_results().to_csv(
          self.inputs.data_path/filename,
          index=False)


# Fluxo Principal

## Importação dos arquivos

In [ ]:
from pathlib import Path
import wget

def load_files():

  filenames = []
  dir = Path("data_colab")

  if dir.exists():
    for a in dir.iterdir():
       a.unlink() if a.is_file() else a.rmdir()
    dir.rmdir()
  dir.mkdir()

  file_ids = ["1j95_pfpNDwzyIl-FgUQ1eeCW_2pW3FbS",
              # "1A79213nSjjOeM-1ZxIP-We04dYTpKTr5",
              ]

  for file_id in file_ids:
    url = f"https://docs.google.com/uc?export=download&id={file_id}"
    # Get filename and extract dir part
    filename = wget.download(url, out=dir.name)[ len(dir.name)+1: ]
    filenames.append(filename)

  return dir, filenames

def load_model():
  dir, filenames = load_files()
  dados = DadosConsumoConsciente("Agua", dir, *filenames)
  modelo = ModeloConsumoConsciente(dados, n_clusters = 10)

  return dados, modelo

## Main

In [ ]:
def main():
    dados_energia, modelo_energia = load_model()
    modelo_energia.save_final_result()
if __name__ ==  "__main__":
   main()

DADOS_PARA_MODELAGEM_AGUA_2025.xlsx
DADOS_PARA_MODELAGEM_AGUA_2025.xlsx
         CENSO  SAIDAS_AGUA  TOTAL_ESTUDANTES  TOTAL_AMBIENTES
0     33000026         47.0             193.0               33
1     33000042         70.0             178.0               53
2     33000115         58.0              99.0               22
3     33000158         53.0             569.0               36
4     33000360        146.0             711.0               36
...        ...          ...               ...              ...
1227  33518211         40.0             234.0               25
1228  33518220         17.0              59.0               17
1229  33518238         85.0             320.0               35
1230  33558221         23.0             266.0               19
1231  33558230         42.0            1087.0               33

[1194 rows x 4 columns]
